In [3]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [93]:
import yfinance as yf
import pandas as pd

csv = pd.read_csv('constituents.csv')

In [94]:
print(list(csv['Symbol'])[:5])

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']


In [37]:
stock_list = list(csv['Symbol'])
stock_information = yf.download(stock_list, start='2022-01-01', end='2024-01-01')

[**********************70%*********              ]  351 of 503 completedCould not get exchangeTimezoneName for ticker 'HWM' reason: 'chart'
[*********************100%***********************]  503 of 503 completed

6 Failed downloads:
['SOLV', 'GEV', 'SW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2024-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1641013200, endDate = 1704085200")')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2024-01-01)')
['BRK.B', 'HWM']: YFTzMissingError('possibly delisted; no timezone found')


In [33]:
print(stock_information.xs('AAPL', level=1, axis=1))

Price            Close        High         Low        Open     Volume
Date                                                                 
2023-01-03  123.632515  129.395502  122.742858  128.782633  112117500
2023-01-04  124.907692  127.181260  123.642405  125.431599   89113600
2023-01-05  123.583107  126.301500  123.326101  125.668857   80962700
2023-01-06  128.130219  128.792516  123.454587  124.561717   87754700
2023-01-09  128.654144  131.876686  128.397138  128.970474   70790800
...                ...         ...         ...         ...        ...
2023-12-22  192.444580  194.243775  191.818335  194.015137   37122800
2023-12-26  191.897873  192.732856  191.679185  192.454528   28919300
2023-12-27  191.997284  192.345201  189.949580  191.341234   48087700
2023-12-28  192.424713  193.498269  192.017156  192.981369   34049900
2023-12-29  191.380966  193.239801  190.585738  192.742785   42628800

[250 rows x 5 columns]


In [95]:
aapl_close = stock_information['Close']['AAPL']
qcom_close = stock_information['Close']['QCOM']
print(aapl_close)
print("\n")
print(qcom_close)

Date
2022-01-03    178.879913
2022-01-04    176.609619
2022-01-05    171.911850
2022-01-06    169.042053
2022-01-07    169.209122
                 ...    
2023-12-22    192.444595
2023-12-26    191.897873
2023-12-27    191.997269
2023-12-28    192.424713
2023-12-29    191.380951
Name: AAPL, Length: 501, dtype: float64


Date
2022-01-03    174.090652
2022-01-04    175.044281
2022-01-05    174.361801
2022-01-06    173.847595
2022-01-07    168.668167
                 ...    
2023-12-22    140.715073
2023-12-26    142.646988
2023-12-27    142.901962
2023-12-28    143.039246
2023-12-29    141.833023
Name: QCOM, Length: 501, dtype: float64


In [102]:
correlation = qcom_close.corr(aapl_close)
print("Correlation: " + f"{correlation}")

Correlation: 0.12945347573403376


In [115]:
close_prices = stock_information['Close']
correlation_matrix = close_prices.corr().iloc[:5, :5]
print(correlation_matrix)

Ticker         A      AAPL      ABBV      ABNB       ABT
Ticker                                                  
A       1.000000 -0.385216  0.236380 -0.119114  0.385753
AAPL   -0.385216  1.000000 -0.102390  0.585909  0.074876
ABBV    0.236380 -0.102390  1.000000 -0.151612 -0.197152
ABNB   -0.119114  0.585909 -0.151612  1.000000  0.558956
ABT     0.385753  0.074876 -0.197152  0.558956  1.000000


In [52]:
# Creates an array of stock pairs (X,Y) with correlation > 0.8

stock_pair = []
n = len(correlation_matrix)
for i in range(n):
    for j in range(n):
        if (correlation_matrix.iloc[i,j] >= 0.8) and (i > j):
            stock_pair.append([correlation_matrix.index[i],correlation_matrix.index[j]])

In [160]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Calculates the spread between 2 stocks

def spread(a,b):

    # Extract price series
    x_series = refined_stock_information['Close'][a]
    y_series = refined_stock_information['Close'][b]
    
    # Create a combined DataFrame to align and drop NaNs
    df = pd.DataFrame({'x': x_series, 'y': y_series}).dropna()
    if len(df) == 0:  # All data is NaN
        return x_series * 0  # Return zeros with same index
    
    # Reshape data for regression
    x = df['x'].values.reshape(-1, 1)
    y = df['y'].values
    
    # Fit regression model
    model = LinearRegression().fit(x, y)
    beta = model.coef_[0]
    
    # Calculate spread for all original indices
    spread = x_series - beta * y_series  # NaNs will remain where data was missing
    return spread

print(spread('AAPL','QCOM')[:5])

Date
2024-01-02    173.535379
2024-01-03    172.359890
2024-01-04    170.146027
2024-01-05    169.376450
2024-01-08    173.549953
dtype: float64


In [134]:
import copy

#Normlalises spread

def normalise(X):
    mean = X.mean()
    std = X.std()
    Z = copy.deepcopy(X)
    for i in range(len(X)):
        Z.iloc[i] = (X.iloc[i]-mean)/std
    return Z
        

In [135]:
print(normalise(spread('AAPL','QCOM'))[:5])

Date
2022-01-03    0.618739
2022-01-04    0.483608
2022-01-05    0.222236
2022-01-06    0.063236
2022-01-07    0.108283
dtype: float64


In [144]:
upper = -float('inf')
lower = float('inf')
for z in normalise(spread('AAPL','QCOM')):
    if z < lower:
        lower = z
    if z > upper:
        upper = z
print(f"Max and min spread: {max}, {min}")

Max and min spread: 1.8933474721304477, -2.0348256117844805


In [86]:
#Downloads stock information for stock canditates

refined_stock_list = []
for [a,b] in stock_pair:
    if a not in refined_stock_list:
        refined_stock_list.append(a)
    if b not in refined_stock_list:
        refined_stock_list.append(b)
refined_stock_information = yf.download(refined_stock_list, start='2024-01-01', end='2025-01-01')

[*********************100%***********************]  451 of 451 completed


In [161]:
#Create hashmap with key stockpair and value Z
Z_hash = {}
for (s,p) in stock_pair:
    Z_hash[(s, p)] = normalise(spread(s, p)).ffill().fillna(0)

In [173]:
#Backtests trading algorithm from 2024 to 2025

time = 0
cash = 0
trades = {} #Tracks active trades with the values indicating amount of stock owned
d = 10 #price we spend on each trade in pounds
min_cash = 0
No_of_Buys = 0
No_of_StopLoss = 0
No_of_Sells = 0

while time < len(Z_hash[list(Z_hash.keys())[0]]) - 1: #inefficient creates full list of keys
    #print(time)
    for (s,p) in stock_pair:
        
        if cash < min_cash:
            min_cash = cash
            
        Z = Z_hash[(s,p)]
        
        if ((s,p) not in trades): #Buy condition
            if ((Z.iloc[time] > 2) or (Z.iloc[time] < -2)):
                No_of_Buys += 1
                if refined_stock_information['Close'][s].iloc[time] <= refined_stock_information['Close'][p].iloc[time]:
                    aos_s = d/refined_stock_information['Close'][s].iloc[time]  #long
                    aos_p = d/refined_stock_information['Close'][p].iloc[time]  #Short
                    trades[(s,p)] = (aos_s,-aos_p) 
                else:
                    aos_p = d/refined_stock_information['Close'][p].iloc[time]  #long
                    aos_s = d/refined_stock_information['Close'][s].iloc[time]  #Short
                    trades[(s,p)] = (-aos_s,aos_p)
                
        elif ((Z.iloc[time] > 2.5) or (Z.iloc[time] < -2.5)): #Stop loss
            No_of_StopLoss += 1
            (aos_s,aos_p) = trades[(s,p)]
            delta_cash = aos_s*refined_stock_information['Close'][s].iloc[time] + aos_p*refined_stock_information['Close'][p].iloc[time]
            cash += delta_cash
            del trades[(s,p)]
            
        elif -0.5 < Z.iloc[time] < 0.5: #Sell condition
            No_of_Sells += 1
            (aos_s,aos_p) = trades[(s,p)]
            delta_cash = aos_s*refined_stock_information['Close'][s].iloc[time] + aos_p*refined_stock_information['Close'][p].iloc[time]
            cash += delta_cash
            del trades[(s,p)]
            
    time += 1

#Liquidate position
time -= 1
for (a,b) in list(trades.keys()):
    values = trades[(a,b)]
    cash += refined_stock_information['Close'][a].iloc[time]*values[0] + refined_stock_information['Close'][b].iloc[time]*values[1]
    del trades[(a,b)]

print(f"Profit: {cash}")
print(f"Required cashflow: {-min_cash}")
print(f"Yearly rate of return: {(-cash/min_cash)*100}%")
print(f"Number of Buys: {No_of_Buys}"
print(f"Number of StopLoss sells: {No_of_StopLoss}")
print(f"Number of successful sells: {No_of_Sells})

Profit: 1401.3117971334261
Required cashflow: 230.4624905574563
Yearly rate of return: 608.0433278942054%
10743 3531 4990


In [ ]:
''assumptions — no transaction costs, perfect liquidity, unlimited shorting, and frictionless execution.
With this volume of trades even a 0.10 fee on each purchase and sale would eliminate profits entirely and result in a loss''
